In [27]:
# imports a library 'pandas', names it as 'pd'

import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV


# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [28]:
read_in_file = pd.read_csv('C:/Users/simon/Desktop/smnioffe/projects/appt1.csv')#, nrows=1000000)

In [29]:
#%reset C:\Users\Simon\Desktop\smnioffe\projects

In [30]:
read_in_file = read_in_file.dropna()
df= read_in_file.copy()
df.head()

,﻿app_month,appt_begintime,weekday,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,kept_perc,...,eth_refused,eth_hispanic,lang_Eng,lang_Chi,lang_Spa,lang_Other,male,female,income,depression
0,8,11,5,1,0,0,0,1,0,0.583333,...,0,0,1,0,0,0,0,1,73052,0
1,6,11,5,1,0,0,0,1,0,0.800000,...,0,0,1,0,0,0,0,1,43329,0
2,4,16,5,0,0,1,1,0,0,0.418181,...,0,0,1,0,0,0,0,1,56657,0
3,4,9,2,0,0,1,1,0,0,0.930232,...,0,0,1,0,0,0,0,1,39741,0
4,1,14,3,1,0,0,1,0,0,0.877551,...,0,0,1,0,0,0,0,1,51756,1


In [31]:
cols = ['month']
for i in range(1,len(df.columns)):
    cols.append(df.columns[i])
df.columns=cols

df = pd.concat([df, pd.get_dummies(df['month'], prefix='month')], axis=1)

In [32]:
df = pd.concat([df, pd.get_dummies(df['weekday'], prefix='wkday')], axis=1)

In [33]:
df = pd.concat([df, pd.get_dummies(df['appt_begintime'], prefix='hour')], axis=1)

In [34]:
df.income = df.income.round(-4)
df = pd.concat([df, pd.get_dummies(df['income'], prefix='inc')], axis=1)

In [35]:
#df['income'] = df.income.map(lambda x: df['income'].median() if (x > 0))

In [36]:
#break out ages into discrete groups

df['age_65_and_over'] = 0
df['age_under_5'] = 0
df['age_5_17'] = 0
df['age_18_29'] = 0
df['age_30_44'] = 0
df['age_45_64'] = 0


df['age_65_and_over'] = df.age.map(lambda x: 1 if (x > 64) else 0)
df['age_under_5'] = df.age.map(lambda x: 1 if (x < 5) else 0)
df['age_5_17'] = df.age.map(lambda x: 1 if (x > 4 and x <18 ) else 0)
df['age_18_29'] = df.age.map(lambda x: 1 if (x > 17 and x <30 ) else 0)
df['age_30_44'] = df.age.map(lambda x: 1 if (x > 29 and x <45 ) else 0)
df['age_45_64'] = df.age.map(lambda x: 1 if (x > 44 and x <65 ) else 0)

df.head()

,month,appt_begintime,weekday,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,kept_perc,...,inc_140000,inc_150000,inc_170000,inc_200000,age_65_and_over,age_under_5,age_5_17,age_18_29,age_30_44,age_45_64
0,8,11,5,1,0,0,0,1,0,0.583333,...,0,0,0,0,0,0,0,1,0,0
1,6,11,5,1,0,0,0,1,0,0.800000,...,0,0,0,0,1,0,0,0,0,0
2,4,16,5,0,0,1,1,0,0,0.418181,...,0,0,0,0,0,0,0,0,1,0
3,4,9,2,0,0,1,1,0,0,0.930232,...,0,0,0,0,0,0,0,0,0,1
4,1,14,3,1,0,0,1,0,0,0.877551,...,0,0,0,0,0,0,0,0,0,1


In [37]:
columnNames=df.columns

In [38]:
from sklearn import preprocessing

xi = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
xi_scaled = min_max_scaler.fit_transform(xi)
df = pd.DataFrame(xi_scaled)


df.columns=columnNames



In [39]:
df.corr()

,month,appt_begintime,weekday,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,kept_perc,...,inc_140000,inc_150000,inc_170000,inc_200000,age_65_and_over,age_under_5,age_5_17,age_18_29,age_30_44,age_45_64
month,1.000000,-0.001963,-0.033692,-0.006344,0.008123,0.000262,-0.006977,0.008968,0.000175,-0.015393,...,0.000872,-0.000282,0.000178,-0.002090,-0.000717,0.007426,0.004960,-0.001755,-0.005306,-0.001644
appt_begintime,-0.001963,1.000000,-0.033780,0.004911,-0.023662,0.016231,-0.010673,-0.004037,0.017467,-0.010492,...,-0.001268,-0.001721,-0.001830,-0.000248,-0.058282,0.046296,0.094564,0.019886,-0.008904,-0.076313
weekday,-0.033692,-0.033780,1.000000,-0.011384,0.011566,0.003319,-0.010462,0.013685,0.000035,-0.014962,...,-0.001680,-0.000019,0.000887,0.001336,-0.010097,0.000871,0.015601,0.003182,-0.000481,-0.009095
kept,-0.006344,0.004911,-0.011384,1.000000,-0.617708,-0.668316,0.071189,-0.047958,-0.043974,0.126755,...,-0.001846,-0.000177,0.001325,-0.000250,0.016392,0.015861,0.008089,-0.010593,-0.015668,-0.002760
rescheduled,0.008123,-0.023662,0.011566,-0.617708,1.000000,-0.172167,-0.046409,0.093236,-0.031358,-0.100204,...,0.002988,-0.000824,-0.001714,0.001663,-0.016595,-0.000514,0.008672,0.004402,0.007157,-0.006797
cancelled,0.000262,0.016231,0.003319,-0.668316,-0.172167,1.000000,-0.045273,-0.028120,0.084744,-0.063990,...,-0.000514,0.001000,-0.000038,-0.001260,-0.004835,-0.019382,-0.018336,0.009106,0.012856,0.009886
last_kept,-0.006977,-0.010673,-0.010462,0.071189,-0.046409,-0.045273,1.000000,-0.630581,-0.659447,0.364130,...,0.000757,0.001311,0.000537,-0.001372,0.017910,0.015071,0.004723,-0.007647,-0.013576,-0.004856
last_rescheduled,0.008968,-0.004037,0.013685,-0.047958,0.093236,-0.028120,-0.630581,1.000000,-0.167617,-0.218988,...,-0.000477,-0.000826,-0.001720,0.001652,-0.017310,-0.001774,0.009127,0.004155,0.004935,-0.003728
last_cancelled,0.000175,0.017467,0.000035,-0.043974,-0.031358,0.084744,-0.659447,-0.167617,1.000000,-0.250415,...,-0.000499,-0.000864,0.000985,0.000144,-0.005983,-0.017426,-0.014840,0.005689,0.012464,0.009780
kept_perc,-0.015393,-0.010492,-0.014962,0.126755,-0.100204,-0.063990,0.364130,-0.218988,-0.250415,1.000000,...,0.001765,-0.000591,0.000688,-0.003096,0.054645,0.028341,0.006326,-0.029615,-0.029939,-0.003223


In [40]:
#df.corr()['kept'].order(ascending=False)

In [41]:
#k=corrVals[corrVals.ix[:1]>.001]
#print(k)

In [42]:
corr_series = df.corr()['kept'].order(ascending=False)
#pd.set_option('display.height', 500)
#pd.set_option('display.max_rows', 500)

print(corr_series)

kept                1.000000
kept_perc           0.126755
last_kept           0.071189
inc_50000           0.025303
hour_18             0.017163
age_65_and_over     0.016392
age_under_5         0.015861
wkday_1             0.014288
inc_30000           0.013246
wkday_2             0.012796
hour_20             0.012175
hour_19             0.011923
wkday_7             0.011879
male                0.011105
hour_9              0.008608
hour_21             0.008099
age_5_17            0.008089
hour_11             0.007607
hour_12             0.006588
month_1             0.006252
appt_begintime      0.004911
month_11            0.004070
month_3             0.003997
lang_Other          0.003803
hour_17             0.003779
month_2             0.002749
eth_not_hispanic    0.002484
month_4             0.001744
hour_10             0.001533
inc_10000           0.001508
                      ...   
hour_15            -0.004469
inc_130000         -0.004470
hour_14            -0.004539
month_9       

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


In [43]:
selected_columns = corr_series[((corr_series > .02 ) | (corr_series < -.02))].index

selected_columns = selected_columns[((selected_columns != 'kept' ) & (selected_columns != 'cancelled') & (selected_columns != 'rescheduled'))]

print(selected_columns)

Index([u'kept_perc', u'last_kept', u'inc_50000', u'income', u'last_cancelled',
       u'last_rescheduled', u'can_perc', u'resch_perc'],
      dtype='object')


In [44]:

X = df.ix[:, selected_columns] 

y = df.ix[:, df.columns == 'kept']




In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [46]:


#np.all(np.isfinite(mat))

In [47]:
#supervised classification
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()

#lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.70398098295758571

In [53]:
#df.info()#0.71724745080587426

In [49]:
read_in_file = read_in_file.dropna()



y_pred = lr.predict(X)



data = pd.DataFrame({"Actual":read_in_file.kept, "Predicted":y_pred.tolist()})



In [50]:
#get calculated coeffcicients
lr.coef_

array([[ 0.8406697 ,  0.1995246 ,  0.08934337, -0.495134  ,  0.06046331,
         0.10173795, -0.19333886, -0.71239172]])

In [51]:
import csv

data_top=data.head(10000)

data_top.to_csv("predictions.csv")

